In [57]:
from datetime import date
import pandas as pd
import numpy as np
import holidays
from connection import connect

In [59]:
SCHEMA='production'
co_oltp,etl_conn,etl_conn_or=connect(SCHEMA)

['product_list_price_history', 'product_model', 'product_model_illustration', 'product_model_prod_desc_culture', 'product_subcategory', 'scrap_reason', 'transaction_history', 'transaction_history_archive', 'bill_of_materials', 'unit_measure', 'document', 'work_order', 'culture', 'illustration', 'location', 'product', 'product_category', 'product_cost_history', 'product_photo', 'product_product_photo', 'product_review', 'product_description', 'work_order_routing', 'product_document', 'product_inventory']


In [65]:
dim_fecha = pd.DataFrame({
    "date": pd.date_range(start='1/1/2005', end='31/12/2014', freq='D')
})


In [66]:
dim_fecha['day_number_of_week'] = ((dim_fecha['date'].dt.day_of_week + 1) % 7) + 1
dim_fecha['day_number_of_month']=dim_fecha['date'].dt.day
dim_fecha['day_number_of_year']=dim_fecha['date'].dt.day_of_year
dim_fecha['week_number_of_year'] = dim_fecha['date'].apply(
    lambda d: ((d - pd.Timestamp(d.year, 1, 1)).days + pd.Timestamp(d.year, 1, 1).weekday() + 1) // 7 + 1
)
dim_fecha['month_number_of_year']=dim_fecha['date'].dt.month
dim_fecha['calendar_quarter']=dim_fecha['date'].dt.quarter
dim_fecha['calendar_year']=dim_fecha['date'].dt.year
dim_fecha['calendar_semester']=(((dim_fecha['date'].dt.month-1)//6)+1)
dim_fecha['fiscal_year']=dim_fecha['date'].apply(lambda d: d.year if d.month <=7 else d.year+1)
dim_fecha['fiscal_month']=dim_fecha['date'].apply(lambda d:((d.month)-7)%12+1)
dim_fecha['fiscal_quarter']=((dim_fecha['fiscal_month']-1)//3) + 1
dim_fecha['fiscal_semester']=((dim_fecha['fiscal_month']-1)//6) + 1

dim_fecha.drop('fiscal_month',inplace=True,axis=1)

#languages


days = {
    1: 'Sunday',
    2: 'Monday',
    3: 'Tuesday',
    4: 'Wednesday',
    5: 'Thursday',
    6: 'Friday',
    7: 'Saturday'
}

dim_fecha['english_day_name_of_week'] = dim_fecha['day_number_of_week'].map(days)

dim_fecha










,date,day_number_of_week,day_number_of_month,day_number_of_year,week_number_of_year,month_number_of_year,calendar_quarter,calendar_year,calendar_semester,fiscal_year,fiscal_quarter,fiscal_semester,english_day_name_of_week
0,2005-01-01,7,1,1,1,1,1,2005,1,2005,3,2,Saturday
1,2005-01-02,1,2,2,2,1,1,2005,1,2005,3,2,Sunday
2,2005-01-03,2,3,3,2,1,1,2005,1,2005,3,2,Monday
3,2005-01-04,3,4,4,2,1,1,2005,1,2005,3,2,Tuesday
4,2005-01-05,4,5,5,2,1,1,2005,1,2005,3,2,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,2014-12-27,7,27,361,52,12,4,2014,2,2015,2,1,Saturday
3648,2014-12-28,1,28,362,53,12,4,2014,2,2015,2,1,Sunday
3649,2014-12-29,2,29,363,53,12,4,2014,2,2015,2,1,Monday
3650,2014-12-30,3,30,364,53,12,4,2014,2,2015,2,1,Tuesday


In [ ]:
dim_fecha['date_key']=dim_fecha['date'].dt.strftime('%Y%m%d').astype(int)


dim_fecha.rename(columns={'date':'full_date_alternate_key'},inplace=True)


dim_fecha

## Load

In [63]:
dim_fecha.to_sql('dim_date',etl_conn,if_exists='append',index=False)

IntegrityError: (psycopg2.errors.NotNullViolation) null value in column "english_day_name_of_week" of relation "dim_date" violates not-null constraint
DETAIL:  Failing row contains (20050101, 2005-01-01, 7, null, null, null, 1, 1, 1, null, null, null, 1, 1, 2005, 1, 3, 2005, 2).

[SQL: INSERT INTO dim_date (full_date_alternate_key, day_number_of_week, day_number_of_month, day_number_of_year, week_number_of_year, month_number_of_year, calendar_quarter, calendar_year, calendar_semester, fiscal_year, fiscal_quarter, fiscal_semester, d ... 354484 characters truncated ... __999)s, %(fiscal_year__999)s, %(fiscal_quarter__999)s, %(fiscal_semester__999)s, %(date_key__999)s)]
[parameters: {'week_number_of_year__0': 1, 'fiscal_year__0': 2005, 'calendar_year__0': 2005, 'fiscal_quarter__0': 3, 'calendar_quarter__0': 1, 'fiscal_semester__0': 2, 'calendar_semester__0': 1, 'date_key__0': 20050101, 'full_date_alternate_key__0': datetime.datetime(2005, 1, 1, 0, 0), 'day_number_of_month__0': 1, 'month_n